## Tradition RAG (Retrieval Augmented Generation)

-   Enhance quality, accuracy and relevance of responses generated by LLMs.
-   Reduces Hallucinations
-   Access to Up-To-Date Information
-   Cost-effective

#### DATA LOADER

In [1]:
from langchain_community.document_loaders import PyPDFLoader
FILE_PATH=r"/Users/vaasubisht/Developer/AgenticAI/vector-database/system-development-lifecycle.pdf"

loader = PyPDFLoader(FILE_PATH)

In [2]:
# sync loading
pages=loader.load()
print(len(pages))

Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)


4


In [3]:
# async loading
pages = []
async for page in loader.alazy_load():
    pages.append(page)

print(len(pages))

Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)


4


#### Chunking

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

split_docs = splitter.split_documents(pages)
len(split_docs)

22

#### Vector database

In [5]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

In [6]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ['GOOGLE_API_KEY']=os.getenv("GOOGLE_API_KEY")


from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
test_embedding = embeddings_model.embed_query("This is a good test")
print(len(test_embedding))

768


In [7]:
index=faiss.IndexFlatIP(768)
vector_store=FAISS(
    embedding_function=embeddings_model,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [8]:
vector_store.add_documents(documents=split_docs)

['afeba7cc-989d-469c-a589-4f423f3f53fa',
 '9d2af3fc-0198-40fa-b2e7-0ecb1355f725',
 'd51352e3-c5a6-4dde-a661-ff42da235054',
 '3aeff2fc-4501-4054-a12e-fccaa0696544',
 '2c94397d-1197-48b3-964f-fae9cef713af',
 'c9c0a65f-d14f-4ec9-8b8c-c4e200f6da37',
 '9d0eab15-1254-42d8-bc8f-97666d6da93d',
 '15704147-de49-4f0c-b762-ddef3c29b8b7',
 'dca06321-9d96-4c82-89cf-4dc4909ec12e',
 '0b5a817d-5a15-4148-97ef-afc77177d18d',
 'd3365411-033b-43aa-9acd-5f96b9285674',
 'e8aa51cf-65b5-4dd4-b17f-e6a087beb84f',
 'fbd1c229-93c3-4e50-961a-df5966e95e38',
 '7368a9e2-5c9b-4a15-affa-f3d77d8b2b92',
 '23f366af-98ef-45b5-9493-1eb7741f0421',
 'e19a6388-8094-461b-a52a-7741353e4ec4',
 '5c950eff-061d-4686-9ab5-d4b878d5eb6a',
 'd9700f5a-c827-4593-a654-e13d1727026e',
 'd8e8439f-5cd7-417a-8192-65b451c46042',
 '99fbbfd1-86cb-45fb-9223-0868db19522e',
 'af0271c2-683a-44a5-acea-3d4305da9453',
 'bc31a9cf-bfa9-443c-b63b-12adcadc347f']

In [9]:
retriever=vector_store.as_retriever(
    search_kwargs={
        "k": 7
    }
)

In [10]:
retriever.invoke("what is a software devlopment lifecycle?")

[Document(id='9d0eab15-1254-42d8-bc8f-97666d6da93d', metadata={'producer': 'Mac OS X 10.11.6 Quartz PDFContext', 'creator': 'Word', 'creationdate': "D:20160920174206Z00'00'", 'title': 'system-development-lifecycle', 'author': 'Amy Blake', 'subject': '', 'moddate': "D:20160920174206Z00'00'", 'keywords': '', 'aapl:keywords': "['']", 'source': '/Users/vaasubisht/Developer/AgenticAI/vector-database/system-development-lifecycle.pdf', 'total_pages': 4, 'page': 1, 'page_label': '2'}, page_content='Guidelines\t&\tProcedures\t\tA\tsoftware\tapplication\ttypically\tundergoes\tseveral\tdevelopment\tlifecycles,\tcorresponding\tto\tits\tcreation\tand\tsubsequent\tupgrades.\tEach\tsuch\tdevelopment\tlifecycle\tconstitutes\ta\tproject.\tSuch\tprojects\tcontinue\tuntil\tthe\tunderlying\ttechnology\tages\tto\tthe\tpoint\twhere\tit\tis\tno\tlonger\teconomical\tto\tinvest\tin\tupgrades\tand\tthe\tapplication\tis\tconsidered\tfor\teither\tcontinued\tas-is\toperation\tor\tretirement.\t\tThe\tSDLC\tPhases\t

#### LLM

In [11]:
# model
from langchain_google_genai import ChatGoogleGenerativeAI
model=ChatGoogleGenerativeAI(model='gemini-1.5-flash')

#prompt
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

In [12]:
prompt.messages

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]

input_variables=['context', 'question']
-   context -> retriever
-  question -> user

In [13]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [14]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [15]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [16]:
rag_chain.invoke("what is a software devlopment lifecycle?")

'A software development lifecycle (SDLC) is a process used to create and upgrade software applications.  It typically involves multiple phases, such as initiation, design, and testing,  and continues until the application is no longer economical to maintain.  Each SDLC constitutes a project with defined work products and documents.'